S08 T01: Tasca Feature Engineering

Nivell 1
- Exercici 1
Agafa un conjunt de dades de tema esportiu que t'agradi i normalitza els atributs categòrics en dummy. Estandaritza els atributs numèrics amb StandardScaler.

Nivell 2 NO
- Exercici 2
Continua amb el conjunt de dades de tema esportiu que t'agradi i aplica l'anàlisi de components principals.

Nivell 3 NO
- Exercici 3
Continua amb el conjunt de dades de tema esportiu que t'agradi i normalitza les dades tenint en compte els outliers.

Conceptes:
    Normalization requires that you know the minimum and maximum values for each attribute.
    
    Standardization is a rescaling technique that refers to centering the distribution of the data on the value 0 and the standard deviation to the value 1.
Together, the mean and the standard deviation can be used to summarize a normal distribution, also called the Gaussian distribution or bell curve.    

In [20]:
import numpy as np
import pandas as pd
from scipy import stats
from math import sqrt
from sklearn.preprocessing import StandardScaler
import dummy as dummy

In [2]:
#Dades sobre fets, denúncies i trencaments de condemna per violència domèstica (Mossos d'Esquadra Open Data)
#https://mossos.gencat.cat/ca/els_mossos_desquadra/indicadors_i_qualitat/dades_obertes/cataleg_dades_obertes/

crime_untidy = pd.read_csv("/Users/fiorelliris/Documents/Coding/PYTHON/Sprint6/FetsMossos.csv")
crime_untidy.head(5)

,Any,Trimestre,Edat,Tipologia,Àmbit,Fet principal sense trencament de condemna,Trencament de condemna,Denúncies
0,2011,1r,>65,Violència domèstica,Persones grans,196,15,138
1,2011,2n,>65,Violència domèstica,Persones grans,286,28,188
2,2011,3r,>65,Violència domèstica,Persones grans,308,27,209
3,2011,4t,>65,Violència domèstica,Persones grans,245,27,181
4,2011,1r,<18,Violència domèstica,Menors,162,6,130


In [25]:
#Renaming columns 
crime_tidy = crime_untidy.rename(columns = {"Fet principal sense trencament de condemna": "Fet_principal", "Trencament de condemna": "Trencament_condemna", "Denúncies": "Denuncies"}, inplace = False)
crime = crime_tidy[["Àmbit","Fet_principal", "Trencament_condemna", "Denuncies"]].copy()
crime.head(5)

,Àmbit,Fet_principal,Trencament_condemna,Denuncies
0,Persones grans,196,15,138
1,Persones grans,286,28,188
2,Persones grans,308,27,209
3,Persones grans,245,27,181
4,Menors,162,6,130


In [48]:
crime.dtypes

Àmbit                  object
Fet_principal           int64
Trencament_condemna     int64
Denuncies               int64
dtype: object

In [51]:
dummy= pd.get_dummies(crime["Àmbit"])
dummy.head()

,Menors,Persones grans
0,0,1
1,0,1
2,0,1
3,0,1
4,1,0


In [53]:
crime = pd.concat([crime, dummy], axis=1)
#merge???
#crime = crime.merge(dummy, left_index = True, right_index = True )
crime.head()

,Àmbit,Fet_principal,Trencament_condemna,Denuncies,Menors,Persones grans
0,Persones grans,196,15,138,0,1
1,Persones grans,286,28,188,0,1
2,Persones grans,308,27,209,0,1
3,Persones grans,245,27,181,0,1
4,Menors,162,6,130,1,0


In [36]:
from sklearn.preprocessing import MinMaxScaler
X= crime_tidy[["Fet_principal", "Trencament_condemna", "Denuncies"]]
mms= MinMaxScaler()
Xn= mms.fit_transform(X)

In [37]:
print(Xn.min(axis=0))
print(Xn.max(axis=0))

[0. 0. 0.]
[1. 1. 1.]


In [50]:
print(Xn.std(axis=0))

[0.22627564 0.22766282 0.25228823]


In [38]:
# Find the min and max values for each column
def dataset_minmax(X):
    minmax = list()
    for i in range(len(X[0])):
        col_values = [row[i] for row in X]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

In [39]:
# Rescale dataset columns to the range 0-1
def normalize_dataset(X, minmax):
    for row in X:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [42]:
# Calculate min and max for each column
minmax = dataset_minmax(X)

KeyError: 0

In [41]:
# Normalize columns
normalize_dataset(X, minmax)
print(X[0])

NameError: name 'minmax' is not defined

In [ ]:
# calculate column means
def column_means(crime):
    means = [0 for i in range(len(crime[0]))]
    for i in range(len(crime[0])):
        col_values = [row[i] for row in crime]
        means[i] = sum(col_values) / float(len(crime))
    return means

Standard deviation:
Describes the average spread of values from the mean. It can be calculated as the square root of the sum of the squared difference between each value and the mean and dividing by the number of values minus 1.

In [ ]:
# calculate column standard deviations
def column_stdevs(crime, means):
    stdevs = [0 for i in range(len(crime[0]))]
    for i in range(len(crime[0])):
        variance = [pow(row[i]-means[i], 2) for row in crime]
        stdevs[i] = sum(variance)
    stdevs = [sqrt(x/(float(len(crime)-1))) for x in stdevs]
    return stdevs

In [ ]:
# Estimate mean and standard deviation
means = column_means(crime)
stdevs = column_stdevs(crime, means)
print(means)
print(stdevs)

In [ ]:
# standardize dataset
def standardize_dataset(crime, means, stdevs):
    for row in crime:
        for i in range(len(row)):
        row[i] = (row[i] - means[i]) / stdevs[i]
        
print(crime)